In [1]:
import torch
from diffusers import CogVideoXPipeline
from diffusers.utils import export_to_video

prompt = "A panda, dressed in a small, red jacket and a tiny hat, sits on a wooden stool in a serene bamboo forest. The panda's fluffy paws strum a miniature acoustic guitar, producing soft, melodic tunes. Nearby, a few other pandas gather, watching curiously and some clapping in rhythm. Sunlight filters through the tall bamboo, casting a gentle glow on the scene. The panda's face is expressive, showing concentration and joy as it plays. The background includes a small, flowing stream and vibrant green foliage, enhancing the peaceful and magical atmosphere of this unique musical performance."
pipe = CogVideoXPipeline.from_pretrained(
    "THUDM/CogVideoX-5b",
    torch_dtype=torch.bfloat16
)

pipe.enable_model_cpu_offload()
pipe.vae.enable_tiling()

video = pipe(
    prompt=prompt,
    num_videos_per_prompt=1,
    num_inference_steps=50,
    num_frames=49,
    guidance_scale=6,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

export_to_video(video, "output.mp4", fps=8)


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

It is recommended to use `export_to_video` with `imageio` and `imageio-ffmpeg` as a backend. 
These libraries are not present in your environment. Attempting to use legacy OpenCV backend to export video. 
Support for the OpenCV backend will be deprecated in a future Diffusers version


'output.mp4'

In [2]:
from datasets import load_dataset
# Replace 'your_dataset_name' with the actual dataset name
dataset = load_dataset("yesidobyte/nsfw1024", split='train')
# Specify the directory to save images
save_dir = '/workspace/data/samples_nudity/images'
import os
import requests
from PIL import Image
from io import BytesIO
# Create the directory if it doesn't exist
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
# Select the first 200 images
first_200_images = dataset.select(range(200))
# Access the images
for i, example in enumerate(first_200_images):
    # Replace 'image_column' with the actual column name containing image URLs or data
    image = example['image']
    try:
        # If the image is a URL, download it
        if isinstance(image, str):
            response = requests.get(image)
            with open(f"{save_dir}/image_{i}.jpg", 'wb') as f:
                f.write(response.content)
        # If the image is a PIL Image object
        elif isinstance(image, Image.Image):
            with open(f"{save_dir}/image_{i}.jpg", 'wb') as f:
                image.save(f, format='JPEG')
    except:
        pass

In [4]:
!pip install accelerate
!pip install modelcards
!pip install transformers
!pip install deepspeed
!pip install diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 123.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 176.0 MB/s eta 0:00:00
  Created wheel for deepspeed: filename=deepspeed-0.15.0-py3-none-any.whl size=1483696 sha256=174a84dc43380fb2328befb689be67c9aa2c131190507e2b9f787b6283f6a50f
  Stored in directory: /root/.cache/pip/wheels/8f/5b/de/79a2a54b4af5d58057e0f4569e6b45c6a853bb404b59007373
Successfully built deepspeed
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 88.3 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.30.2
    Uninstalling diffusers-0.30.2:
      Successfully uninstalled diffusers-0.30.2


In [1]:
with open('/workspace/data/samples_nudity/caption.txt', 'w') as file:
    for _ in range(200):
        file.write('naked woman\n')

In [2]:
!cd /workspace/concept-ablation/diffusers  && accelerate launch train.py \
          --pretrained_model_name_or_path="/workspace/CogVideoX-5b"  \
          --output_dir="/workspace/log_ouput" \
          --class_data_dir="/workspace/data/samples_nudity" \
          --class_prompt="people, body"  \
          --caption_target "nudity, nsfw" \
          --concept_type nudity \
          --resolution=512  \
          --train_batch_size=4  \
          --learning_rate=2e-7  \
          --max_train_steps=4 \
          --scale_lr --hflip \
          --parameter_group full-weight \
          --enable_xformers_memory_efficient_attention

/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")
/workspace/concept-ablation/diffusers/train.py:541: UserWarning: You need not use --class_data_dir without --with_prior_preservation.
  warnings.warn(
/workspace/concept-ablation/diffusers/train.py:545: UserWarning: You need not use --class_prompt without --with_prior_preservation.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate